# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf
import joblib
warnings.filterwarnings(action='ignore')

# 모델,스케일러,데이터 객체 로드

In [2]:
with open('./models/forward/forward_qt.pickle','rb') as f:
    forward_po = pickle.load(f)

forward_model = load_model('./models/forward/forward_nn_high.h5')

premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


# 데이터 변환

In [4]:
forward_df = premier_df[premier_df['포지션']=='공격수']
forward_df = forward_df.reset_index(drop=True)

# Goal_Shoot_linear
forward_df['GSL'] = (forward_df['경기당 유효 슈팅/90']+0.5) + (forward_df['경기당 슈팅/90']) + (forward_df['골']*1.3)

# PSTL = Pass sucess try linear
forward_df['PSTL'] = (forward_df['경기당 패스 성공/90']) + (forward_df['경기당 패스 시도/90'])

# CACCL = Cross Attemp Cross Complete Linear
forward_df['CACCL'] = (forward_df['크로스 시도/90']) + (forward_df['크로스 성공률'])


# 독립 변수, 종속 변수 상세
independent_value = {'CACCL','GSL','기회 창출/90','도움','드리블 성공/90','태클 성공/90','파울 획득','헤더 성공/90'}

dependent_value = {'평균획득승점/경기'}

# 독립변수 스케일링

In [9]:
forward_df.loc[:,independent_value] = forward_po.transform(forward_df.loc[:,independent_value])

# PPG 예측

In [11]:
forward_df['예측PPG']=forward_model.predict(forward_df.loc[:,independent_value])

# WAR 계산 (PPG - Min_PPG) + 1

In [13]:
forward_min = np.min(forward_df['예측PPG'].tolist())
forward_df['WAR'] = forward_df['예측PPG'].map(lambda x : x-forward_min+1)

2.3768210411071777

# 독립변수를 알아볼 수 있게 다시 복원

In [15]:
forward_df.loc[:,independent_value] = forward_po.inverse_transform(forward_df.loc[:,independent_value])

# WAR 확인

In [16]:
forward_war = forward_df.sort_values(by='WAR',ascending=False).copy()
forward_war = forward_war.reset_index(drop=True)

In [17]:
forward_war[['이름','시즌','골', '도움','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,시즌,골,도움,WAR
0,Jadon Sancho,2021-2022,15,14.0,2.188231
1,Harry Kane,2021-2022,25,12.0,2.140416
2,Mohamed Salah,2021-2022,19,8.0,2.135593
3,Heung-Min Son,2021-2022,21,6.0,2.131160
4,Cristiano Ronaldo,2021-2022,21,9.0,2.073582
5,Raheem Sterling,2021-2022,10,6.0,2.058792
6,Kai Havertz,2021-2022,22,8.0,2.050762
7,Chris Wood,2021-2022,22,1.0,2.045957
8,Sadio Mané,2021-2022,13,4.0,2.044212
9,Odsonne Edouard,2021-2022,21,1.0,2.024609
